Load data for network from file

In [135]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from pprint import pprint
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np



In [136]:
wine = np.genfromtxt("datasets/wine/wine_train_array.csv", delimiter=',')


In [137]:
wine[0]

array([0.59210526, 0.17786561, 0.79569892, 0.25257732, 0.43478261,
       0.55862069, 0.68823529, 0.42      , 0.29746835, 0.28327645,
       0.49593496, 0.56981132, 0.48666128, 1.        , 0.        ,
       0.        ])

In [138]:
X = wine[:,:-3]
Y = wine[:,-3:]

In [139]:
X[0]

array([0.59210526, 0.17786561, 0.79569892, 0.25257732, 0.43478261,
       0.55862069, 0.68823529, 0.42      , 0.29746835, 0.28327645,
       0.49593496, 0.56981132, 0.48666128])

In [140]:
Y[0]

array([1., 0., 0.])

In [141]:
#train_x,train_y,test_x,test_y = create_feature_sets_and_labels('Data/pos.txt','Data/neg.txt')
train_x = X
train_y = Y
n_nodes_hl1 = 5
n_nodes_hl2 = 5

n_classes = len(Y[0])
hm_epochs = 10


In [142]:
class TfAnn(object):
    
    def __init__(self):
        self.hidden_1_layer = {'weights':[],'biases':[]}
        self.hidden_2_layer = {'weights':[],'biases':[]}
        self.output_layer = {'weights':[],'biases':[]}
        self.np_hidden_1_layer={"weights":[],"biases":[]}
        self.np_hidden_2_layer={"weights":[],"biases":[]}
        self.np_output_layer={"weights":[],"biases":[]}
    
    def init_empty(self,size,n_nodes_hl1,n_nodes_hl2,n_classes):
        print(size,n_nodes_hl1,n_nodes_hl2,n_classes)
        self.hidden_1_layer = {'weights':tf.Variable(tf.random_normal([size, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

        self.hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
        
        self.output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                        'biases':tf.Variable(tf.random_normal([n_classes]))}


    def init_values(self,size,l1_weights,l2_weights,l3_weights,out_weights,l1_biases,l2_biases,l3_biases,out_biases):
        self.hidden_1_layer = {'weights':tf.Variable(l1_weights),'biases':tf.Variable(l1_biases)}
        self.hidden_2_layer = {'weights':tf.Variable(l2_weights),'biases':tf.Variable(l2_biases)}
        self.output_layer = {'weights':tf.Variable(out_weights),'biases':tf.Variable(out_biases)}


    def create(self,data):
        # This is the heart of the ann where multiply the data by the wights to the layers 
        l1 = tf.add(tf.matmul(data,self.hidden_1_layer['weights']), self.hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)

        l2 = tf.add(tf.matmul(l1,self.hidden_2_layer['weights']), self.hidden_2_layer['biases'])
        l2 = tf.nn.relu(l2)

        
        output =  tf.add(tf.matmul(l2,self.output_layer['weights']) , self.output_layer['biases'])

        return output
    
    def extract(self):
        self.np_hidden_1_layer["weights"] = neural_network_model.hidden_1_layer["weights"].eval()
        self.np_hidden_2_layer["weights"] = neural_network_model.hidden_2_layer["weights"].eval()
        self.np_output_layer["weights"] = neural_network_model.output_layer["weights"].eval()
        self.np_hidden_1_layer["biases"] = neural_network_model.hidden_1_layer["biases"].eval()
        self.np_hidden_2_layer["biases"] = neural_network_model.hidden_2_layer["biases"].eval()
        self.np_output_layer["biases"] = neural_network_model.output_layer["biases"].eval()
        

In [143]:
def train_neural_network_interactive(neural_network_model,fold):
    x= tf.placeholder('float')
    y = tf.placeholder('float')
    prediction = neural_network_model.create(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 1500
    sess = tf.InteractiveSession()
    init = tf.initialize_all_variables()
    sess.run(init)
    for epoch in range(hm_epochs):
            epoch_loss = 0
            _, c = sess.run([optimizer, cost], feed_dict={x: train_x[train_index[fold]],
                                                          y: train_y[train_index[fold]]})
            epoch_loss += c
            #if epoch % 100 == 0:
            #    print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

    neural_network_model.extract()
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:',accuracy.eval({x:train_x[test_index[fold]], y:train_y[test_index[fold]]}))
    y_test=(sess.run(prediction,feed_dict={
                             x: train_x[test_index[fold]]
                              }))       
    
    true_class=np.argmax(train_y[test_index[fold]],1)
    predicted_class=np.argmax(y_test,1)
    cm=confusion_matrix(predicted_class,true_class)
    pprint(cm)

In [144]:
from sklearn.model_selection import StratifiedKFold
train_index=[]
test_index=[]
splits =10
skf = StratifiedKFold(n_splits=splits,random_state=0,shuffle=True)
for train_idx, test_idx in skf.split(train_x,train_y[:,0]):               
    train_index.append(train_idx)
    test_index.append(test_idx)

In [145]:
for i in range(10):
    neural_network_model = TfAnn()
    neural_network_model.init_empty(len(train_x[0]),n_nodes_hl1,n_nodes_hl2,n_classes)
    train_neural_network_interactive(neural_network_model,i)

13 5 5 3


/home/ric/Projects/Python/env/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Accuracy: 0.8666667
array([[4, 1, 0],
       [1, 5, 0],
       [0, 0, 4]])
13 5 5 3
Accuracy: 0.53333336
array([[2, 1, 1],
       [3, 6, 2],
       [0, 0, 0]])
13 5 5 3
Accuracy: 0.93333334
array([[4, 0, 0],
       [1, 3, 0],
       [0, 0, 7]])
13 5 5 3
Accuracy: 0.93333334
array([[5, 0, 0],
       [0, 5, 1],
       [0, 0, 4]])
13 5 5 3
Accuracy: 0.93333334
array([[5, 1, 0],
       [0, 6, 0],
       [0, 0, 3]])
13 5 5 3
Accuracy: 1.0
array([[5, 0, 0],
       [0, 6, 0],
       [0, 0, 3]])
13 5 5 3
Accuracy: 1.0
array([[5, 0, 0],
       [0, 4, 0],
       [0, 0, 5]])
13 5 5 3
Accuracy: 1.0
array([[4, 0, 0],
       [0, 6, 0],
       [0, 0, 3]])
13 5 5 3
Accuracy: 0.9230769
array([[4, 0, 0],
       [0, 6, 0],
       [0, 1, 2]])
13 5 5 3
Accuracy: 0.9230769
array([[4, 0, 0],
       [0, 6, 1],
       [0, 0, 2]])
